In [2]:
import os
import re
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as skm
import piexif
import pytesseract
pytesseract.pytesseract.tesseract_cmd = "/opt/homebrew/bin/tesseract"


from os import PathLike
from IPython.display import display
from matplotlib.patches import Patch
from matplotlib.gridspec import GridSpec
from scipy.stats import spearmanr
from pathlib import Path
from sklearn.metrics import ConfusionMatrixDisplay
from typing import cast, Tuple
from random import randint
from PIL import Image
from PIL.ExifTags import TAGS

from src.dataset import MammaliaDataSequence, MammaliaDataImage
from src.transform import ImagePipeline, BatchImagePipeline
from src.utils import load_path_yaml
from src.eval_helpers import *

set_custom_plot_style()
cm = 1 / 2.54

export = True

eval = MammaliaEval(
    path_config='../data/path_config.yml',
    metrics=['balanced_accuracy_score'],
    force_metrics=False
    )

Loading pre-calculated metrics from file...
